# LOG6302A - Lab6

CESAR Pierre 2310298

MERSALI Bilal 2309771


In [ ]:
!git clone https://github.com/CesarPierr/LOG6302.git
import os
os.chdir("LOG6302/lab6")

### 1 : Fragment le plus dupliqué

In [1]:
from collections import defaultdict
from code_analysis import ASTReader
import numpy as np
from code_analysis.ASTFragmentation import AST_fragmentation
from code_analysis.AST import AST

In [2]:
def count_nodes(ast):
    return len(ast.get_node_ids())

def find_largest_group(ast_dir):
    vect_groups = defaultdict(list)
    
    for root, _, files in os.walk(ast_dir):
        for file in files:
            if ".ast.json.gz" in file:
                file_path = os.path.join(root, file)
                reader = ASTReader()
                ast = reader.read_ast(file_path)
                
                if count_nodes(ast) > 100:
                    vect = tuple(ast.vectorize())
                    vect_groups[vect].append(file_path)
    
    max_group = max(vect_groups.values(), key=len)
    return max_group


In [3]:
ast_dir = "ast"
largest_group = find_largest_group(ast_dir)
print(f"Le plus gros groupe de fichiers avec des vecteurs identiques contient {len(largest_group)} fichiers.")
print(f"Le plus gros groupe de fichiers avec des vecteurs identiques est : ")
for file in largest_group :
    print(file)


Le plus gros groupe de fichiers avec des vecteurs identiques contient 47 fichiers.
Le plus gros groupe de fichiers avec des vecteurs identiques est : 
ast/4417/rabo2020/def/pin/index.php.ast.json.gz
ast/4417/rabo2020/def/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/5b311143a3244728d432f81fe755d7aa/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/5b311143a3244728d432f81fe755d7aa/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/5b311143a3244728d432f81fe755d7aa/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/2284e439f95ccee06d62fa62fb0f1b6f/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/9bfe1de0245319b7c25a09cc5d44ece1/start/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/9bfe1de0245319b7c25a09cc5d44ece1/info/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/9bfe1de0245319b7c25a09cc5d44ece1/cc/index.php.ast.json.gz
ast/2099/dhl/a1b2c3/082e3739ba7c548

#### Analyse du résultat
Le plus gros groupe de fichiers avec des vecteurs identiques contient 47 fichiers provenant de deux kits de phishing différents :

`ast/4417/rabo2020/...` : Ce kit cible probablement la banque Rabo en 2020 car l contient des pages de phishing pour collecter les informations de carte bancaire (pin, info).
`ast/2099/dhl/...` : Ce kit se fait passer pour le service de livraison DHL. Il contient de nombreuses variantes du même kit, avec des pages pour collecter les informations de livraison (start), les informations personnelles (info) et les données de carte bancaire (cc).
Le fait que ces fichiers aient des vecteurs identiques indique qu'ils partagent exactement la même structure de code, malgré le changement de cible (Rabo, DHL) et de champs collectés.

Cela suggère fortement que ces kits ont été générés par un outil automatisé à partir d'un même code source, ou qu'un kit a été copié et modifié superficiellement pour changer le thème mais en gardant le code sous-jacent.

La présence de nombreuses variantes du kit DHL avec des hashs différents (5b311143a3244728d432f81fe755d7aa, 2284e439f95ccee06d62fa62fb0f1b6f, etc) mais un code identique renforce cette hypothèse d'une génération automatisée avec un paramétrage assez basique.

### 1.2

In [4]:
def find_similar_group(ast_dir, threshold=0.3):
    vector_groups = defaultdict(list)
    vectors = []
    file_paths = []
    
    for root, dirs, files in os.walk(ast_dir):
        for file in files:
            if file.endswith(".ast.json.gz"):
                file_path = os.path.join(root, file)
                reader = ASTReader()
                ast = reader.read_ast(file_path)
                
                if count_nodes(ast) > 100:
                    vector = ast.vectorize()
                    vectors.append(vector)
                    file_paths.append(file_path)
    
    for i, vector_i in enumerate(vectors):
        similar_vectors = [file_paths[i]]
        for j, vector_j in enumerate(vectors):
            if i != j and np.abs(vector_j - vector_i).sum() <= threshold * np.sum(vector_i):
                similar_vectors.append(file_paths[j])
        vector_groups[tuple(vector_i)].append(similar_vectors)
        
    largest_group = max(vector_groups.values(), key=lambda x: len(x[0]))
    return largest_group[0]

In [5]:
ast_dir = "ast"  
largest_similar_group = find_similar_group(ast_dir)
print(f"Le plus gros groupe de fichiers avec des vecteurs similaires contient {len(largest_similar_group)} fichiers.")
print(f"Le plus gros groupe de fichiers avec des vecteurs similaires est : ")
for file in largest_similar_group :
    print(file)


Le plus gros groupe de fichiers avec des vecteurs similaires contient 81 fichiers.
Le plus gros groupe de fichiers avec des vecteurs similaires est : 
ast/2110/moneylion/def/token/index.php.ast.json.gz
ast/2110/moneylion/def/token/index - Copy.php.ast.json.gz
ast/2110/moneylion/def/wifi/index.php.ast.json.gz
ast/2110/moneylion/def/read/index.php.ast.json.gz
ast/2110/moneylion/def/token2/index.php.ast.json.gz
ast/2110/moneylion/def/token3/index.php.ast.json.gz
ast/2110/moneylion/def/cc/index.php.ast.json.gz
ast/2110/moneylion/def/login/index.php.ast.json.gz
ast/4417/rabo2020/a1b2c3/8ed3cd96687bb07c77db299054948df1/token/index.php.ast.json.gz
ast/4417/rabo2020/a1b2c3/8ed3cd96687bb07c77db299054948df1/done/index.php.ast.json.gz
ast/4417/rabo2020/a1b2c3/8ed3cd96687bb07c77db299054948df1/login/index.php.ast.json.gz
ast/4417/rabo2020/a1b2c3/44a24055e71e417ccebb479ccef7bacf/token/index.php.ast.json.gz
ast/4417/rabo2020/a1b2c3/44a24055e71e417ccebb479ccef7bacf/done/index.php.ast.json.gz
ast/4417/

#### Analyse
Le plus gros groupe de fichiers avec des vecteurs similaires contient 81 fichiers provenant de trois kits de phishing différents :

`ast/2110/moneylion/...` : Ce kit cible probablement l'application MoneyLion. Il contient des pages de phishing pour collecter les informations de connexion (login), les tokens d'authentification (token, token2, token3), les informations de carte bancaire (cc), et d'autres données (wifi, read).
`ast/4417/rabo2020/...` : Ce kit cible la banque Rabo en 2020, comme vu précédemment. Ici on voit des variantes (`a1b2c3/8ed3cd96687bb07c77db299054948df1/`, `a1b2c3/44a24055e71e417ccebb479ccef7bacf/`) qui ont une structure similaire avec des pages pour la connexion (login), la saisie du token (token) et la confirmation (done).
ast/2099/dhl/... : Le kit se faisant passer pour DHL, également vu précédemment. Cette fois-ci avec une page supplémentaire vbv probablement pour la "Verified by Visa".
Malgré les cibles et les champs différents, ces kits partagent une structure de code très similaire, avec seulement des variations mineures qui maintiennent les vecteurs de métriques dans les 30% de distance.

Cela renforce l'hypothèse d'un outil de génération paramétrable et/ou de la réutilisation massive de code entre les différents kits, quelles que soient leurs cibles. Les auteurs de kits semblent travailler par copier-coller et modifications légères plutôt que par réécriture complète à chaque kit.

## Partie 2
### 2.1

In [6]:
def get_subtree(ast : AST, node_id):
    subtree_nodes = ast.dfs(node_id)
    subtree = AST()
    for node in subtree_nodes:
        subtree.set_type(node, ast.get_type(node))
        subtree.set_image(node, ast.get_image(node))
        subtree.set_position(node, ast.get_position(node))
        for child in ast.get_children(node):
            if child in subtree_nodes:
                subtree.add_edge(node, child)
    return subtree

def find_similar_fragments(ast_dir, min_size=10, threshold=0.1):
    vector_groups = defaultdict(list)
    kit_fragments = defaultdict(set)
    
    for root, _, files in os.walk(ast_dir):
        kit = os.path.dirname(root)
        for file in files:
            if ".ast.json.gz" in file:
                file_path = os.path.join(root, file)
                reader = ASTReader()
                ast = reader.read_ast(file_path)
                
                for fragment_root in AST_fragmentation(ast):
                    fragment = get_subtree(ast, fragment_root)
                    if count_nodes(fragment) > min_size:
                        vector = tuple(fragment.vectorize()) #tuple pour pouvoir l'utiliser comme clé de dictionnaire
                        if vector not in kit_fragments[kit]:
                            kit_fragments[kit].add(vector)
                            for group in vector_groups[vector]:
                                if kit not in [x[0] for x in group]:
                                    group.append((kit, file_path, fragment_root))
                                    break
                            else:
                                vector_groups[vector].append([(kit, file_path, fragment_root)])
                                
    max_group = max(vector_groups.values(), key=lambda x: len(x))
    
    # Filtrer les groupes pour ne garder que les fragments similaires
    similar_groups = []
    for group in max_group:
        for i, (_, _, root_i) in enumerate(group):
            for j, (_, _, root_j) in enumerate(group):
                if i != j:
                    vector_i = np.array(vector_groups[tuple(get_subtree(ast, root_i).vectorize())])
                    vector_j = np.array(vector_groups[tuple(get_subtree(ast, root_j).vectorize())])
                    if np.abs(vector_j - vector_i).sum() <= threshold * np.sum(vector_i):
                        similar_groups.append(group)
                        break
            else:
                continue
            break
    
    return similar_groups



In [7]:
ast_dir = "ast"
similar_fragments = find_similar_fragments(ast_dir)
print(f"Les fragments les plus similaires sont : {similar_fragments}")


WARN: Type None unknown. Skip node 889648965..
WARN: Type None unknown. Skip node 754078534..
Les fragments les plus similaires sont : [[('ast/2620/absa', 'ast/2620/absa/php/sqlite.php.ast.json.gz', 889648965), ('ast/2141/index', 'ast/2141/index/php/sqlite.php.ast.json.gz', 754078534)]]


Le fait que les fragments similaires soient trouvés dans des fichiers nommés 'sqlite.php' dans deux kits différents montre que ces kits partagent du code lié et notamment pour l'interaction avec des bases de données ce qui pourrait indiquer un meme acteur derriere qui recupere les données

### 2.2

In [8]:
# Pour trouver le nom de la fonction correspondante
for group in similar_fragments:
    for kit, file_path, root_id in group:
        reader = ASTReader()
        ast = reader.read_ast(file_path)
        function_name = ast.get_type(root_id)
        image_name = ast.get_image(root_id)
        print(f"Fragment dans {file_path}, fonction : {function_name}, image : {image_name}")

Fragment dans ast/2620/absa/php/sqlite.php.ast.json.gz, fonction : MethodStatement, image : __construct
Fragment dans ast/2141/index/php/sqlite.php.ast.json.gz, fonction : MethodStatement, image : __construct


Comme les fragments correspondent à des définitions de constructeurs __construct on peut imaginer que les kits utilisent une classe similaire ou partagée pour gérer les connexions ou les requêtes SQLite.


## 3. Kits parametriques

In [9]:
def compute_kit_vectors(ast_dir):
    kit_vectors = {}
    for root, _, files in os.walk(ast_dir):
        kit = os.path.dirname(root)
        kit_vector = np.zeros(126)  # En supposant une taille de vecteur de 126
        for file in files:
            if ".ast.json.gz" in file:
                file_path = os.path.join(root, file)
                reader = ASTReader()
                ast = reader.read_ast(file_path)
                for fragment_root in AST_fragmentation(ast):
                    fragment = get_subtree(ast, fragment_root)
                    kit_vector += fragment.vectorize()
        kit_vectors[kit] = kit_vector
    return kit_vectors

def find_identical_kits(kit_vectors):
    vector_groups = defaultdict(list)
    for kit, vector in kit_vectors.items():
        vector_groups[tuple(vector)].append(kit)
    largest_group = max(vector_groups.values(), key=len)
    return largest_group

# Utilisation
ast_dir = "ast"
kit_vectors = compute_kit_vectors(ast_dir)
identical_kits = find_identical_kits(kit_vectors)
print(f"Les kits ayant des vecteurs identiques sont : {identical_kits}")

# Vérification des différences de types et d'images
for kit in identical_kits:
    print(f"Analyse du kit : {kit}")
    kit_dir = os.path.join(ast_dir, kit)
    for root, _, files in os.walk(kit_dir):
        for file in files:
            if ".ast.json.gz" in file:
                file_path = os.path.join(root, file)
                reader = ASTReader()
                ast = reader.read_ast(file_path)
                print(f"  Fichier : {file}")
                for node_id in ast.get_node_ids():
                    node_type = ast.get_type(node_id)
                    node_image = ast.get_image(node_id)
                    print(f"    Nœud {node_id} - Type : {node_type}, Image : {node_image}")

ValueError: operands could not be broadcast together with shapes (126,) (127,) (126,) 